In [1]:
import numpy as np
import pandas as pd
import itertools
import cma
import os
import sys
import argparse
import pickle
import random
import re
from pprint import pprint

In [3]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit import Aer
from qiskit import IBMQ
from qiskit.compiler import transpile
from qiskit.providers.aer.noise.noise_model import NoiseModel
from qiskit.test.mock import *
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
import mitiq

In [ ]:
IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
print("provider:", provider)

In [ ]:
real_device = provider.get_backend('ibmq_jakarta')

In [ ]:
def qrem_encoder(num_qubits: int, initial_layout: list) -> list:
    qr = QuantumRegister(num_qubits)
    meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
    return transpile(meas_calibs, initial_layout=initial_layout, basis_gates=["sx", "rz", "cx"])

In [ ]:
def execute_circuits(qcs: list, backend, ) -> (qiskit.providers.Job, str):
    job = execute(qcs, backend=backend, )
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

In [ ]:
def qrem_decoder():
    

# ZNE Circuit

In [ ]:
def TwirlCircuit(circ: str) -> QuantumCircuit:
    """
    そのまま使う: 修正は後回し
    """
    #! qasm ベタ書き
    def apply_pauli(num: int, qb: int) -> str:
        if (num == 0):
            return f'id q[{qb}];\n'
        elif (num == 1):
            return f'x q[{qb}];\n'
        elif (num == 2):
            return f'y q[{qb}];\n'
        else:
            return f'z q[{qb}];\n'

    paulis = [(i,j) for i in range(0,4) for j in range(0,4)]
    paulis.remove((0,0))
    paulis_map = [(0, 1), (3, 2), (3, 3), (1, 1), (1, 0), (2, 3), (2, 2), (2, 1), (2, 0), (1, 3), (1, 2), (3, 0), (3, 1), (0, 2), (0, 3)]

    new_circ = ''
    ops = circ.qasm().splitlines(True) #! 生のqasmコードを持ってきてる: オペレータに分解
    for op in ops:
        if (op[:2] == 'cx'): # can add for cz, etc.
            num = random.randrange(len(paulis)) #! permute paulis
            qbs = re.findall('q\[(.)\]', op)
            new_circ += apply_pauli(paulis[num][0], qbs[0])
            new_circ += apply_pauli(paulis[num][1], qbs[1])
            new_circ += op
            new_circ += apply_pauli(paulis_map[num][0], qbs[0])
            new_circ += apply_pauli(paulis_map[num][1], qbs[1])
        else:
            new_circ += op
    return qiskit.circuit.QuantumCircuit.from_qasm_str(new_circ)

In [ ]:
def zne_wrapper(qcs, scale_factors = [1.0, 2.0, 3.0]):
    """
    """
    folded_qcs = [] #! ZNE用の回路
    for qc in qcs:
        folded_qcs.append([mitiq.zne.scaling.fold_gates_at_random(qc, scale) for scale in scale_factors]) #! ここでmitiqを使用
    folded_qcs = list(itertools.chain(*folded_qcs)) #! folded_qcsを平坦化
    folded_qcs = [Twirlqc(circ) for circ in folded_qcs] #! 後からPauli Twirlingを施す!
    return folded_qcs

In [ ]:
def zne_decoder(expvals: list, scale_factors = [1.0, 2.0, 3.0]) -> list:
    """
    """
    zero_noise_values = []
    if isinstance(backend, qiskit.providers.aer.backends.qasm_simulator.QasmSimulator): # exact_sim
        for i in range(len(expvals)):
            zero_noise_values.append( np.mean(expectation_values[i * len(scale_factors): (i + 1) * len(scale_factors)]) )
    else: # device_sim, real_device
        fac = mitiq.zne.inference.LinearFactory(scale_factors)
        for i in range(len(expvals)):
            zero_noise_values.append( fac.extrapolate(scale_factors, expectation_values[i * len(scale_factors): (i + 1) * len(scale_factors)]) )
    
    return zero_noise_values

In [ ]:
def trotter_circuit():
    

In [ ]:
# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid